# `flask_sqlalchemy` Tutorial
## Part 1/2, Create the database

In [3]:
import pandas as pd
import datetime as dt
from database_app import db, WalliStat, Campaign

In [4]:
db.create_all()

## Create a Campaign
The default `hourly` Campaign starts the next full hour using the `id=0`

In [5]:
t = dt.datetime.now()    
next_hour = t.replace(second=0, microsecond=0, minute=0, hour=t.hour+1)
hourly = Campaign(id=0, title="hourly", start=next_hour, interval=dt.timedelta(seconds=3600))
hourly

Campaign(id:0, 'hourly' is active:None, start:2021-12-10 22:00:00, end:None, interval:1:00:00)

In [6]:
db.session.add(hourly)
db.session.commit()

## Commit WalliStats from .csv-file
use the default Campaign: `hourly`

In [7]:
hourly.id

0

In [8]:
fn = "ExampleData_2021-07-25.csv"
date_str = fn.split(".")[0].split("_")[1]
df = pd.read_csv(fn)
df.head(7)

,time,ver,charge_state,I_L1,I_L2,I_L3,Temp,V_L1,V_L2,V_L3,...,E_cyc_lb,E_hb,E_lb,I_max,I_min,watchdog,standby,remote_lock,max_I_cmd,FailSafe_I
0,01:01:53,264,4,0,0,0,275,238,236,240,...,0,2,49607,10,6,15000,4,1,0,10
1,01:01:58,264,4,0,0,0,275,238,236,240,...,0,2,49607,10,6,15000,4,1,0,10
2,01:02:15,264,5,0,0,0,275,237,236,240,...,0,2,49607,10,6,15000,4,1,100,10
3,01:02:20,264,7,0,0,0,275,237,236,240,...,0,2,49607,10,6,15000,4,1,100,10
4,01:02:25,264,7,0,0,22,275,238,236,240,...,0,2,49607,10,6,15000,4,1,100,10
5,01:02:30,264,7,53,0,15,275,237,236,240,...,0,2,49607,10,6,15000,4,1,100,10
6,01:02:36,264,7,85,69,92,275,236,235,238,...,5,2,49612,10,6,15000,4,1,100,10


In [9]:
for index, row in df.head(6).iterrows():
    ws = WalliStat(datetime=pd.to_datetime(date_str + " " + row.time).to_pydatetime(), 
                   Temp=row.Temp/10., 
                   Power=row.P,
                   campaign_id=hourly.id)
    db.session.add(ws)
db.session.commit()

In [10]:
WalliStat.query.filter(WalliStat.Temp>1).all()

[WalliStat(id:1-->campaign.id:0, 2021-07-25 01:01:53: 27.5°C, 0W),
 WalliStat(id:2-->campaign.id:0, 2021-07-25 01:01:58: 27.5°C, 0W),
 WalliStat(id:3-->campaign.id:0, 2021-07-25 01:02:15: 27.5°C, 0W),
 WalliStat(id:4-->campaign.id:0, 2021-07-25 01:02:20: 27.5°C, 0W),
 WalliStat(id:5-->campaign.id:0, 2021-07-25 01:02:25: 27.5°C, 531W),
 WalliStat(id:6-->campaign.id:0, 2021-07-25 01:02:30: 27.5°C, 1641W)]

## Create a second Campaign

In [11]:
campaign1 = Campaign(title="High frequency polling for error checking.", start=dt.datetime.now(), end=dt.datetime.now()+dt.timedelta(days=1), interval=dt.timedelta(seconds=1))
db.session.add(campaign1)
db.session.commit()

In [12]:
Campaign.query.all()

[Campaign(id:0, 'hourly' is active:True, start:2021-12-10 22:00:00, end:None, interval:1:00:00),
 Campaign(id:1, 'High frequency polling for error checking.' is active:True, start:2021-12-10 21:38:20.358488, end:2021-12-11 21:38:20.358488, interval:0:00:01)]

In [13]:
vars(campaign1)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1de86cc6a60>,
 'is_active': True,
 'title': 'High frequency polling for error checking.',
 'start': datetime.datetime(2021, 12, 10, 21, 38, 20, 358488),
 'previous': None,
 'interval': datetime.timedelta(seconds=1),
 'measure_light': True,
 'id': 1,
 'end': datetime.datetime(2021, 12, 11, 21, 38, 20, 358488),
 'measure_walli': True}

## Commit WalliStats to the new Campaign
This time, use the `campaign` attribute. Note this hasn't even been defined within the ``database_app.py`` Models.

In [14]:
for index, row in df.tail(3).iterrows():
    ws = WalliStat(datetime=pd.to_datetime(date_str + " " + row.time).to_pydatetime(), 
                   Temp=row.Temp/10., 
                   Power=row.P,
                   campaign=campaign1)
    db.session.add(ws)
db.session.commit()

In [33]:
WalliStat.query.all()

[WalliStat(id:1-->campaign.id:0, 2021-07-25 01:01:53: 27.5°C, 0W),
 WalliStat(id:2-->campaign.id:0, 2021-07-25 01:01:58: 27.5°C, 0W),
 WalliStat(id:3-->campaign.id:0, 2021-07-25 01:02:15: 27.5°C, 0W),
 WalliStat(id:4-->campaign.id:0, 2021-07-25 01:02:20: 27.5°C, 42W),
 WalliStat(id:5-->campaign.id:0, 2021-07-25 01:02:25: 27.5°C, 531W),
 WalliStat(id:6-->campaign.id:0, 2021-07-25 01:02:30: 27.5°C, 1641W),
 WalliStat(id:7-->campaign.id:1, 2021-07-25 10:23:31: 32.8°C, 0W),
 WalliStat(id:8-->campaign.id:1, 2021-07-25 10:23:36: 32.8°C, 0W),
 WalliStat(id:9-->campaign.id:1, 2021-07-25 10:23:41: 32.8°C, 0W)]

## Modify one value in the Database

In [34]:
db.session.query(WalliStat).filter(WalliStat.id == 4).update({"Power": 43})
db.session.commit()